In [ ]:
import pandas as pd
from collections import Counter

# Load the dataset
file_path = "aggregated_daily_data_with_date_aug2023_aug2024.csv"
data = pd.read_csv(file_path)

# Helper function to parse kg data into a dictionary for analysis
def parse_kg_data(kg_column, items_column):
    items = items_column.split(";")
    quantities = kg_column.split(";")
    return {item.strip(): int(qty) for item, qty in zip(items, quantities) if qty.isdigit()}

# Function to generate weekly report with most and least consumed dishes
def generate_weekly_report(data):
    weekly_summary = {}

    # Loop through each row to accumulate consumption data for each week
    for _, row in data.iterrows():
        week_key = f"{row['month_year']}_{row['week']}"
        
        # Parse the kg columns for each meal
        breakfast_data = parse_kg_data(row['breakfast_kg'], row['breakfast_items'])
        lunch_data = parse_kg_data(row['lunch_kg'], row['lunch_items'])
        dinner_data = parse_kg_data(row['dinner_kg'], row['dinner_items'])
        
        # Initialize week in summary if not already present
        if week_key not in weekly_summary:
            weekly_summary[week_key] = {'breakfast': Counter(), 'lunch': Counter(), 'dinner': Counter()}
        
        # Update counters for each meal
        weekly_summary[week_key]['breakfast'].update(breakfast_data)
        weekly_summary[week_key]['lunch'].update(lunch_data)
        weekly_summary[week_key]['dinner'].update(dinner_data)
    
    # Display weekly summary
    for week, meals in weekly_summary.items():
        print(f"\n{week}:\n")

        # Most and least consumed dishes for each meal
        for meal, items in meals.items():
            print(f"Most consumed dishes (in kg) - {meal.capitalize()}:")
            for item, qty in items.most_common(2):  # Top 2 most consumed items
                print(f"{item}: {qty} kg")
            
            print(f"\nLeast consumed dishes (in kg) - {meal.capitalize()}:")
            for item, qty in items.most_common()[:-3:-1]:  # Bottom 2 least consumed items
                print(f"{item}: {qty} kg")
            print()

# Function to generate monthly report with most and least consumed dishes
def generate_monthly_report(data):
    monthly_summary = {}

    # Loop through each row to accumulate consumption data for each month
    for _, row in data.iterrows():
        month_year = row['month_year']
        
        # Parse the kg columns for each meal
        breakfast_data = parse_kg_data(row['breakfast_kg'], row['breakfast_items'])
        lunch_data = parse_kg_data(row['lunch_kg'], row['lunch_items'])
        dinner_data = parse_kg_data(row['dinner_kg'], row['dinner_items'])
        
        # Initialize month in summary if not already present
        if month_year not in monthly_summary:
            monthly_summary[month_year] = {'breakfast': Counter(), 'lunch': Counter(), 'dinner': Counter()}
        
        # Update counters for each meal
        monthly_summary[month_year]['breakfast'].update(breakfast_data)
        monthly_summary[month_year]['lunch'].update(lunch_data)
        monthly_summary[month_year]['dinner'].update(dinner_data)
    
    # Display monthly summary
    for month, meals in monthly_summary.items():
        print(f"\n{month}:\n")

        # Most and least consumed dishes for each meal
        for meal, items in meals.items():
            print(f"Most consumed dishes (in kg) - {meal.capitalize()}:")
            for item, qty in items.most_common(2):  # Top 2 most consumed items
                print(f"{item}: {qty} kg")
            
            print(f"\nLeast consumed dishes (in kg) - {meal.capitalize()}:")
            for item, qty in items.most_common()[:-3:-1]:  # Bottom 2 least consumed items
                print(f"{item}: {qty} kg")
            print()

# Example usage
print("Monthly Summary Report:")
generate_monthly_report(data)

print("\nWeekly Summary Report:")
generate_weekly_report(data)
